<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=16fa0d22112eb5b4193e450c4833c738dd2d8af9626660b2489236ada556d22e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 10:00:13
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 24.96 K (0.18%)
Current PnL: -25.41 L (-16.41%)
CY Booked + Current PnL: -11.12 L (-7.18%)
-------------------
Total profit:  1.63 L
Total loss:  -27.04 L
-------------------
Total Booked + Current PnL: 15.86 L (12.41%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.04 L (63.57%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.49,-18.42,22.69,0.09,18655.0,-18561.0,82216.0,90.82,41.0,M-SC,2.37,253.0,-0.99,0.58,8.01,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.30,8.26,10.56,19.69,21921.0,15843.0,207585.0,-2.32,54.0,X-MC,2.47,78.0,0.72,1.47,15.62,XY25,NTT,AC
50,MASFIN,398.61,0.63,-2.65,25.37,22.05,24199.0,-2595.0,95385.0,-15.99,53.0,H-SC,6.31,164.0,-0.11,0.68,38.65,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.25,-15.95,19.00,0.02,27876.0,-27842.0,146717.0,123.77,43.0,M-SC,12.08,234.0,-1.00,1.04,27.62,OX40N,NTT,PAINTS
52,NESTLEIND,1377.00,-0.02,10.81,9.56,21.40,28117.0,28689.0,294115.0,2.77,56.0,X-LC,5.48,12.0,1.02,2.09,18.17,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,4.90,-38.50,119.96,35.27,83768.0,-43719.0,69830.0,-53.72,53.0,L-SC,9.68,270.0,-0.52,0.50,70.91,XR,NTT,HOTELS
79,VAIBHAVGBL,521.00,3.46,-25.01,114.39,60.77,156781.0,-45717.0,137058.0,58.13,53.0,H-SC,4.72,159.0,-0.29,0.97,23.43,XR,NTT,JEWELLERY
17,COALINDIA,484.83,2.76,-2.40,22.15,19.23,32528.0,-3608.0,146853.0,14.04,71.0,L-LC,2.08,179.0,-0.11,1.04,18.37,XY25,ATH,MINING
80,VALIANTORG,838.00,2.55,-43.23,206.79,74.15,192048.0,-70734.0,92871.0,-267.19,51.0,H-SC,22.46,149.0,-0.37,0.66,19.05,XR,NTT,CHEMICALS
60,REPCOHOME,880.00,2.14,-13.33,119.01,89.82,295516.0,-38193.0,248312.0,-56.78,50.0,H-SC,5.60,160.0,-0.13,1.76,31.38,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-3.42,-56.96,132.51,0.07,78233.0,-78138.0,59039.0,2064.14,70.0,L-SC,3.62,268.0,-1.00,0.42,46.83,OX40N,NTT,JEWELLERY
22,DIXON,18931.72,-3.19,-14.46,47.39,26.07,79134.0,-28236.0,166985.0,-58.39,33.0,X-MC,11.39,56.0,-0.36,1.18,4.00,X40N,ATH,IT
41,INFY,1972.00,-1.66,13.11,18.69,34.24,66144.0,41012.0,353900.0,-12.06,67.0,X-LC,5.79,2.0,0.62,2.51,22.48,X40,NTT,IT
67,SONACOMS,806.63,-1.24,-15.69,65.51,39.54,55871.0,-15873.0,85286.0,-34.10,49.0,M-SC,3.86,220.0,-0.28,0.60,20.26,AR,ATH,AUTO
55,QUESS,424.00,-1.19,-29.35,102.17,42.84,46926.0,-19077.0,45929.0,-53.72,46.0,X-SC,34.26,83.0,-0.41,0.33,3.91,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.26,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,58.0,M-SC,11.87,216.0,-0.01,1.04,9.11,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.26,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,58.0,M-SC,11.87,216.0,-0.01,1.04,9.11,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.25,-15.95,19.00,0.02,27876.0,-27842.0,146717.0,123.77,43.0,M-SC,12.08,234.0,-1.00,1.04,27.62,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.49,-18.42,22.69,0.09,18655.0,-18561.0,82216.0,90.82,41.0,M-SC,2.37,253.0,-0.99,0.58,8.01,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,0.14,-21.22,67.60,32.03,70205.0,-27981.0,103854.0,-14.69,64.0,H-SC,9.07,148.0,-0.40,0.74,16.33,OX40N,NTT,IT
66,SIS,528.00,-0.37,-23.79,59.11,21.26,50020.0,-26410.0,84622.0,1996.89,52.0,H-SC,4.71,166.0,-0.53,0.60,14.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.47,11.32,54.75,72.27,103271.0,19175.0,188623.0,-4.26,74.0,M-LC,3.68,99.0,0.19,1.34,18.19,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.47,11.32,54.75,72.27,103271.0,19175.0,188623.0,-4.26,74.0,M-LC,3.68,99.0,0.19,1.34,18.19,XR,NTT,IT
25,FINCABLES,1641.55,-0.26,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,58.0,M-SC,11.87,216.0,-0.01,1.04,9.11,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.95,-3.78,114.85,106.72,136295.0,-4664.0,118672.0,-52.34,41.0,H-SC,4.29,139.0,-0.03,0.84,18.71,AR,ATH,MISC
6,ASIANTILES,137.00,-1.11,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.21,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.51,-2.68,39.36,35.63,80279.0,-5613.0,203961.0,-16.30,44.0,H-MC,2.40,119.0,-0.07,1.45,13.55,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-3.19,-14.46,47.39,26.07,79134.0,-28236.0,166985.0,-58.39,33.0,X-MC,11.39,56.0,-0.36,1.18,4.00,X40N,ATH,IT
8,AWL,485.00,-0.17,-23.63,100.99,53.50,232723.0,-71309.0,230442.0,-62.99,34.0,X-MC,5.38,58.0,-0.31,1.63,2.35,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.08,-38.79,120.90,35.20,95215.0,-49915.0,78755.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,1.02,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.56,-17.39,88.20,55.47,177265.0,-42305.0,200981.0,-26.75,35.0,X-MC,15.17,64.0,-0.24,1.43,2.98,X40N,ATH,FINANCE
34,HONAUT,58357.33,0.09,-17.41,73.53,43.32,98913.0,-28352.0,134520.0,-29.05,35.0,X-SC,5.62,90.0,-0.29,0.95,3.29,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.34,0.21,24.22,24.48,61127.0,528.0,252384.0,-80.87,50.0,X-SC,1.15,86.0,0.01,1.79,23.40,X40N,NTT,MISC
12,BERGEPAINT,680.0,1.09,-2.37,24.08,21.14,53448.0,-5379.0,221960.0,-9.34,49.0,X-MC,1.19,74.0,-0.10,1.57,25.11,XY24,NTT,PAINTS
20,DABUR,735.0,0.06,-3.42,48.79,43.70,116896.0,-8478.0,239590.0,-11.44,36.0,X-MC,2.09,72.0,-0.07,1.70,11.12,XY24,BTT,FMCG
43,ITC,452.0,0.35,-0.96,11.85,10.78,28205.0,-2303.0,238015.0,-41.15,52.0,X-LC,2.12,5.0,-0.08,1.69,4.23,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.25,6.11,13.81,20.77,34311.0,14310.0,248454.0,-3.08,55.0,X-MC,2.16,62.0,0.42,1.76,13.45,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.16,-25.96,121.71,64.15,214429.0,-61771.0,176180.0,-56.75,38.0,X-SC,3.81,82.0,-0.29,1.25,0.59,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.08,-38.79,120.90,35.20,95215.0,-49915.0,78755.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,1.02,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.38,-19.65,76.11,41.51,161064.0,-51745.0,211620.0,-4.32,42.0,X-MC,7.71,61.0,-0.32,1.50,1.38,XY25,ATH,FMCG
59,RELAXO,1176.00,0.60,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,44.0,X-SC,7.25,91.0,-0.48,0.54,1.87,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.17,-23.63,100.99,53.50,232723.0,-71309.0,230442.0,-62.99,34.0,X-MC,5.38,58.0,-0.31,1.63,2.35,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.19,-29.35,102.17,42.84,46926.0,-19077.0,45929.0,-53.72,46.0,X-SC,34.26,83.0,-0.41,0.33,3.91,XY24,NTT,MISC
59,RELAXO,1176.00,0.60,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,44.0,X-SC,7.25,91.0,-0.48,0.54,1.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.08,-38.79,120.90,35.20,95215.0,-49915.0,78755.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,1.02,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.58,-0.91,25.22,24.08,31724.0,-1160.0,125790.0,-8.86,48.0,X-SC,6.11,89.0,-0.04,0.89,18.52,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.09,-17.41,73.53,43.32,98913.0,-28352.0,134520.0,-29.05,35.0,X-SC,5.62,90.0,-0.29,0.95,3.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.61,-9.14,32.85,20.71,112895.0,-34569.0,343668.0,-21.68,68.0,X-LC,3.46,1.0,-0.31,2.44,14.83,X40,ATH,IT
41,INFY,1972.00,-1.66,13.11,18.69,34.24,66144.0,41012.0,353900.0,-12.06,67.0,X-LC,5.79,2.0,0.62,2.51,22.48,X40,NTT,IT
75,TMPV,600.00,0.46,-26.05,66.30,22.98,110754.0,-58848.0,167050.0,-23.08,51.0,X-LC,6.44,3.0,-0.53,1.18,5.07,XY24,NTT,AUTO
81,VBL,671.64,-0.65,-2.54,39.20,35.67,120671.0,-8007.0,307835.0,-14.05,58.0,X-LC,3.60,4.0,-0.07,2.18,10.97,X40N,ATH,FMCG
43,ITC,452.00,0.35,-0.96,11.85,10.78,28205.0,-2303.0,238015.0,-41.15,52.0,X-LC,2.12,5.0,-0.08,1.69,4.23,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.57,-37.13,111.39,32.89,54064.0,-28670.0,48536.0,-699.18,56.0,L-MC,6.82,259.0,-0.53,0.34,33.66,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-3.42,-56.96,132.51,0.07,78233.0,-78138.0,59039.0,2064.14,70.0,L-SC,3.62,268.0,-1.00,0.42,46.83,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.11,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.21,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
50,MASFIN,398.61,0.63,-2.65,25.37,22.05,24199.0,-2595.0,95385.0,-15.99,53.0,H-SC,6.31,164.0,-0.11,0.68,38.65,XR,ATH,FINANCE
13,BSOFT,831.70,1.30,-17.33,82.39,50.79,95803.0,-24373.0,116280.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.82,35.39,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.30,-17.33,82.39,50.79,95803.0,-24373.0,116280.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.82,35.39,XR,ATH,IT
56,RAJESHEXPO,518.00,-3.42,-56.96,132.51,0.07,78233.0,-78138.0,59039.0,2064.14,70.0,L-SC,3.62,268.0,-1.00,0.42,46.83,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,0.78,-45.21,208.54,69.05,152040.0,-60159.0,72907.0,-55.56,57.0,H-SC,20.95,140.0,-0.40,0.52,12.44,SR,ATH,IT
6,ASIANTILES,137.00,-1.11,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.21,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
84,WIPRO,420.00,-0.47,11.32,54.75,72.27,103271.0,19175.0,188623.0,-4.26,74.0,M-LC,3.68,99.0,0.19,1.34,18.19,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.80
2,50,76.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
MC    30.74
LC    24.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.00
X40N     12.96
XR        9.92
XY25      9.09
AR        8.92
OX40N     7.59
SR        1.01
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.24
H-SC    23.79
X-LC    20.76
M-SC    11.62
X-SC     8.02
H-MC     4.80
L-SC     1.56
M-MC     1.36
M-LC     1.34
H-LC     1.11
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-7.43,41.98
IT,13.15,-13.36,70.37
FINANCE,9.52,-18.82,70.50
MISC,7.27,-26.69,78.52
ELECTRICAL,5.97,-11.56,52.42
PAINTS,5.79,-13.00,29.79
INSURANCE,4.75,-1.41,36.11
PHARMA,4.06,-2.59,34.89
AUTO,2.84,-30.58,75.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3148429.0,21
AR,1315262.0,10
XR,1262663.0,13
X40,980735.0,14
X40N,857343.0,9
OX40N,723032.0,10
XY25,360607.0,6
SR,277903.0,2
MH,78408.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3624738.0,25
M-SC,1419394.0,15
X-MC,1407641.0,16
X-LC,798522.0,11
X-SC,788598.0,8
H-MC,402450.0,3
L-SC,241695.0,3
M-LC,103271.0,1
H-LC,77328.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1266680.0      6
           AR         901510.0      5
M-SC       XY24       808629.0      6
H-SC       XR         769818.0      7
X-MC       X40        484211.0      7
           XY24       403067.0      3
           X40N       337378.0      4
H-SC       OX40N      330419.0      4
M-SC       OX40N      314380.0      5
X-LC       X40        306027.0      5
X-SC       X40N       305022.0      3
           XY24       293079.0      3
H-SC       SR         277903.0      2
X-LC       X40N       214943.0      2
H-MC       AR         212087.0      2
X-SC       X40        190497.0      2
H-MC       XY24       190363.0      1
X-LC       XY24       186611.0      2
X-MC       XY25       182985.0      2
M-SC       XR         172048.0      2
L-SC       XR         163462.0      2
M-SC       AR         124337.0      2
M-LC       XR         103271.0      1
X-LC       XY25        90941.0      2
H-SC       MH          78408.0      1
L-SC       OX40N       78233.0      1
H-LC       AR          77328.0      1
M-MC       XY25        54153.0      1
L-MC       XR          54064.0      1
L-LC       XY25        32528.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
